In [1]:
import pandas as pd
import scipy.spatial as sp
from numpy import inf
import numpy as np
from itertools import takewhile

In [2]:
def k_neighbours(main_dataframe_row, second_dataframe,k, radius_meters):
    coord = second_dataframe[['latitude','longitude']]
    t = sp.cKDTree(coord)# https://gis.stackexchange.com/questions/382899/getting-scipy-ckdtree-to-return-everything-with-a-given-meter-radius
    #  0.1 m approximately equals to 0.0000009 degree.
    m_1 = 10*0.0000009
    dist, idx = t.query([main_dataframe_row.get('latitude'), main_dataframe_row.get('longitude')], second_dataframe.shape[0], distance_upper_bound=m_1*radius_meters)
    idx = idx[dist != inf][0:k] #array
    dist = dist[dist != inf][0:k] #array
    dist = np.round(dist /0.000009,1)
    return idx.tolist(), dist.tolist()

In [3]:
df = pd.read_csv("AB_data_clean.csv")

## Extract features from Trees dataset

In [4]:
df_trees = pd.read_csv("trees_data_clean.csv")

In [ ]:
#DO NOT DELETE OR EXECUTE
df_test = df
distances_trees = df_test.apply(lambda x: pd.Series(k_neighbours(x, df_trees, round(df_trees.shape[0]), 2500), index=['df_trees index_2500m', 'trees distance_2500m']), axis=1)
distances_trees['number_of_trees_2500m'] = distances_trees['trees distance_2500m'].apply(lambda x: len(x))

distances_trees['number_of_trees_1000m'] = distances_trees['trees distance_2500m'].apply(lambda x: len(list(takewhile(lambda y: y<1000, x))))
distances_trees['number_of_trees_500m'] = distances_trees['trees distance_2500m'].apply(lambda x: len(list(takewhile(lambda y: y<500, x))))

distances_trees.drop(['df_trees index_2500m', 'trees distance_2500m'], axis = 1).to_csv('trees_distances_simple.csv')

In [ ]:
#FILE IS LOADED INSTEAD
distances_trees = pd.read_csv("trees_distances_simple.csv")
distances_trees.drop(columns='Unnamed: 0', inplace=True)
distances_trees.head(2)

## Extract features from Rats dataset

In [ ]:
df_rats = pd.read_csv("rats_data_clean.csv")

In [ ]:
#DO NOT DELETE OR EXECUTE
df_test = df
distances_rats = df_test.apply(lambda x: pd.Series(k_neighbours(x, df_rats, round(df_rats.shape[0]), 2500), index=['df_rats index_2500m', 'rats distance_2500m']), axis=1)
distances_rats['number_of_rats_2500m'] = distances_rats['rats distance_2500m'].apply(lambda x: len(x))

distances_rats['number_of_rats_1000m'] = distances_rats['rats distance_2500m'].apply(lambda x: len(list(takewhile(lambda y: y<1000, x))))
distances_rats['number_of_rats_500m'] = distances_rats['rats distance_2500m'].apply(lambda x: len(list(takewhile(lambda y: y<500, x))))

distances_rats.drop(['df_rats index_2500m', 'rats distance_2500m'], axis = 1).to_csv('rats_distances_simple.csv')

In [ ]:
#FILE IS LOADED INSTEAD
distances_rats = pd.read_csv("rats_distances_simple.csv")
distances_rats.drop(columns='Unnamed: 0', inplace=True)
distances_rats.head(2)

## Extract features from Touristic Places dataset

In [ ]:
df_places = pd.read_csv("places_data_clean.csv")

In [ ]:
#DO NOT DELETE OR EXECUTE
df_test = df
distances_places = df_test.apply(lambda x: pd.Series(k_neighbours(x, df_places, round(df_places.shape[0]), 2500), index=['df_places index_2500m', 'places distance_2500m']), axis=1)
distances_places['number_of_places_2500m'] = distances_places['places distance_2500m'].apply(lambda x: len(x))

distances_places['number_of_places_1000m'] = distances_places['places distance_2500m'].apply(lambda x: len(list(takewhile(lambda y: y<1000, x))))
distances_places['number_of_places_500m'] = distances_places['places distance_2500m'].apply(lambda x: len(list(takewhile(lambda y: y<500, x))))

distances_places.drop(['df_places index_2500m', 'places distance_2500m'], axis = 1).to_csv('places_distances_simple.csv')

In [ ]:
#FILE CAN BE LOADED INSTEAD
distances_places = pd.read_csv("places_distances_simple.csv")
distances_places.drop(columns='Unnamed: 0', inplace=True)
distances_places.head(2)

## Merging

In [ ]:
distances_join = distances_places.join([distances_rats, distances_trees], how = 'inner')
df_join  = df.join(distances_join, how = 'inner')
print("Length of AirBnB dataframe: ", df.shape[0])
print("Length of joined distances: ", distances_join.shape[0])
df_join.head(2)

In [ ]:
df_join.to_csv("joined_data.csv")